In [2]:
import json
from pprint import pprint

In [3]:
with open('example_data/doc3.json') as doc:
    x = json.load(doc)
    #print(x)
    #pprint(x)
    # class name
    class_name_html = ('<p style="text-align: center;"><span class="gliffy-placeholder-text" '
'style="font-family: Arial; font-size: 12px; font-weight: bold; '
'text-decoration: none; line-height: 14px; color: rgb(0, 0, '
'0);">{}</span></p>').format('newClassName')
    # class name container
    #pprint(x['stage']['objects'][0]['children'][0])
    
    # attr container
    #pprint(x['stage']['objects'][0]['children'][1])
    
    # class
    #pprint(x['stage']['objects'][0]['children'])
    # classes
    #pprint(x['stage']['objects'])
    x['stage']['objects'][0]['children'][0]['children'][0]['graphic']['Text']['html'] = class_name_html
    #pprint(x['stage']['objects'][0]['children'][0]['children'][0]['graphic']['Text']['html'])
    with open('example_data/doc4.json', 'w') as out:
        json.dump(x, out)
    

In [72]:
with open('example_data/2classes.json') as doc:
    x = json.load(doc)
    #print(x)
    #pprint(x)
    # class name
    # class name container
    #pprint(x['stage']['objects'][0]['children'][0])
    
    # attr container
    #pprint(x['stage']['objects'][0]['children'][1])
    
    # class
    #pprint(x['stage']['objects'][0]['children'])
    # classes
    cls = x['stage']['objects'][0]
    # class self id 3 = 0
    pprint(cls['id'])
    
    #### Class
    print('---- class ids')
    child = cls['children'][0]
    # id = 1
    pprint(child['id'])
    #class_id 1 = 2
    pprint(child['children'][0]['id'])
    # class id 2 = 2 
    pprint(child['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'])

    #### Attr
    print('---- attr ids')
    child = cls['children'][1]
    #pprint(child)
    # id = 3
    pprint(child['id'])
    #class_id 1 = 4
    pprint(child['children'][0]['id'])
    # class id 2 = 4
    pprint(child['constraints']['constraints'][1]['HeightConstraint']['heightInfo'][0]['id'])
    
    #### Methods
    print('---- meth ids')
    child = cls['children'][2]
    # id = 5
    pprint(child['id'])
    #class_id 1 = 6
    pprint(child['children'][0]['id'])
    # class id 2 = 0
    pprint(child['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'])

    
    #all ids
    pprint(cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'])
    
    #pprint(cls)
    
    #pprint(cls['children'][0]['children'][0]['id'])
    #x['stage']['objects'][0]['children'][0]['children'][0]['graphic']['Text']['html'] = class_name_html
    #pprint(x['stage']['objects'][0]['children'][0]['children'][0]['graphic']['Text']['html'])
    #with open('example_data/doc4.json', 'w') as out:
    #    json.dump(x, out)

0
---- class ids
1
2
2
---- attr ids
3
4
4
---- meth ids
5
6
0
[{'id': 1, 'magnitude': 1},
 {'id': 3, 'magnitude': 1},
 {'id': 6, 'magnitude': 1}]


In [82]:
from copy import deepcopy

def create_class(template, name, attr, method, cls_num=0):
    cls = deepcopy(template)
    current_id = cls_num
    # class self id 3 = 0
    cls['id'] = current_id
    #### Class
    child = cls['children'][0]
    # id = 1
    child['id'] = current_id + 1
    #class_id 1 = 2
    child['children'][0]['id'] = current_id + 2
    # class id 2 = 2 
    child['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 2

    #### Attr
    child = cls['children'][1]
    # id = 3
    child['id'] = current_id + 3
    #class_id 1 = 4
    child['children'][0]['id'] = current_id + 4
    # class id 2 = 4
    child['constraints']['constraints'][1]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 4

    child['constraints']['constraints'][0]['PositionConstraint']['nodeId'] = current_id + 1
    
    #### Methods
    child = cls['children'][2]
    # id = 5
    child['id'] = current_id + 5

    
    #class_id 1 = 6
    child['children'][0]['id'] = current_id + 6
    # class id 2 = 0
    child['constraints']['constraints'][1]['PositionConstraint']['nodeId'] = current_id + 3

    
    #all ids
    cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 1
    cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 3
    cls['constraints']['constraints'][0]['HeightConstraint']['heightInfo'][0]['id'] = current_id + 6
    
    class_name_html = ('<p style="text-align: center;"><span class="gliffy-placeholder-text" '
'style="font-family: Arial; font-size: 12px; font-weight: bold; '
'text-decoration: none; line-height: 14px; color: rgb(0, 0, '
'0);">{}</span></p>').format(name)
    cls['children'][0]['children'][0]['graphic']['Text']['html'] = class_name_html
    
    attr_html = ('<p style="text-align: left;"><span class="gliffy-placeholder-text" style="font-family: Arial; '
                'font-size: 12px; font-weight: normal; text-decoration: none; line-height: 14px; color: '
                 'rgb(0, 0, 0);">{}</span></p>').format('<br>'.join(attr))
    cls['children'][1]['children'][0]['graphic']['Text']['html'] = attr_html
    
    
    methods_html = ('<p style="text-align: left;"><span class="gliffy-placeholder-text" style="font-family: Arial; '
                'font-size: 12px; font-weight: normal; text-decoration: none; line-height: 14px; color: '
                 'rgb(0, 0, 0);">{}</span></p>').format('<br>'.join(method))
    cls['children'][2]['children'][0]['graphic']['Text']['html'] = methods_html
    
    
    cls['x'] = 100 + cls_num*30
    return cls
    

with open('example_data/2classes.json') as doc:
    x = json.load(doc)
    cls = x['stage']['objects'][0]
    new_cls1 = create_class(cls, 'class1', ['attr12', 'attr11'], ['method12', 'method11'], 0)
    new_cls2 = create_class(cls, 'class2', ['attr22', 'attr21'], ['method22'], 7)
    x['stage']['objects'] = [new_cls1, new_cls2]
    with open('example_data/output.json', 'w') as out:
        json.dump(x, out)